### import needed libraries

In [46]:
import pandas as pd
import numpy as np

### read in .csv file

In [47]:
df = pd.read_csv('rr_data.csv', low_memory=False)
df.head(10)

,Reporting Railroad Name,Maintenance Railroad Name,Date,Accident Type,Hazmat Released Cars,Persons Evacuated,State Name,Weather Condition,Train Speed,Gross Tonnage,...,Remote Control Locomotive Code,Positive Alcohol Tests,Positive Drug Tests,Passengers Transported,Equipment Damage Cost,Primary Accident Cause,Hours Engineers On Duty,Total Persons Killed,Class,Equipment Type Code
0,Northern Indiana Commuter Transportation District,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,06/06/2017,Derailment,0.0,149.0,ILLINOIS,Clear,10.0,0.0,...,0.0,0.0,0.0,Yes,98627.0,Switch point worn or broken,1.0,0.0,No,3
1,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,06/06/2017,Derailment,0.0,0.0,ILLINOIS,Clear,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Switch point worn or broken,NaN,0.0,No,NaN
2,Conrail,Conrail,04/06/1981,Side collision,0.0,0.0,NEW YORK,Snow,0.0,0.0,...,NaN,0.0,0.0,No,4000.0,Cars left foul,NaN,0.0,1L,5
3,Illinois Central Railroad Company,Cargill Elevator,01/11/2007,Derailment,0.0,0.0,LOUISIANA,Cloudy,4.0,14100.0,...,0.0,NaN,NaN,No,5220.0,Switch damaged or out of adjustment,10.0,0.0,1L,1
4,Illinois Central Railroad Company,Illinois Central Railroad Company,10/07/2017,Hwy-rail crossing,0.0,0.0,ILLINOIS,Clear,0.0,3482.0,...,0.0,NaN,NaN,No,13000.0,Highway user deliberately disregarded crossing...,8.0,0.0,Cl,1
5,Norfolk Southern Railway Company,Norfolk Southern Railway Company,10/15/2017,Derailment,0.0,0.0,ILLINOIS,Rain,3.0,6706.0,...,0.0,0.0,0.0,No,7418.0,"Shoving movement, absence of man on or at lead...",2.0,0.0,CL,1
6,Union Pacific Railroad Company,Union Pacific Railroad Company,12/04/2017,Obstruction,0.0,0.0,ILLINOIS,Cloudy,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Object or equipment on or fouling track - othe...,NaN,0.0,CL,NaN
7,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,12/19/2016,Other (describe in narrative),0.0,0.0,ILLINOIS,Clear,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,Other miscellaneous causes (Provide detailed d...,NaN,0.0,No,NaN
8,Indiana Harbor Belt Railroad Company,Indiana Harbor Belt Railroad Company,03/30/2017,Derailment,0.0,0.0,ILLINOIS,Rain,10.0,2399.0,...,0.0,NaN,NaN,NaN,390870.0,Broken Rail - Base [Desc.Changed-Refer to Curr...,2.0,0.0,Cl,7
9,BNSF Railway Company,BNSF Railway Company,04/06/2017,Derailment,0.0,0.0,ILLINOIS,Clear,10.0,15760.0,...,0.0,0.0,0.0,No,750.0,Damaged flange or tread (build up),9.0,0.0,CL,1


### drop columns that I don't need 

In [48]:
df.drop(['Class', 'Hours Engineers On Duty', 'Maintenance Railroad Name', 'Equipment Attended'], axis=1, inplace=True)


### Check DataFrame dtypes

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215849 entries, 0 to 215848
Data columns (total 17 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Reporting Railroad Name         215846 non-null  object 
 1   Date                            215847 non-null  object 
 2   Accident Type                   215847 non-null  object 
 3   Hazmat Released Cars            215848 non-null  float64
 4   Persons Evacuated               215848 non-null  float64
 5   State Name                      215848 non-null  object 
 6   Weather Condition               215454 non-null  object 
 7   Train Speed                     215846 non-null  float64
 8   Gross Tonnage                   215848 non-null  float64
 9   Remote Control Locomotive Code  54315 non-null   object 
 10  Positive Alcohol Tests          76380 non-null   float64
 11  Positive Drug Tests             76750 non-null   float64
 12  Passengers Trans

### Rename columns

In [50]:
df.rename(columns= {'Equipment Type Code': 'Equipment', 
'Weather Condition': 'Weather', 
'Train Speed': 'Speed', 
'Reporting Railroad Name': 'Railroad', 
'State Name': 'State', 
'Equipment Damage Cost': 'Damage',
'Primary Accident Cause': 'Cause',
'Remote Control Locomotive Code': 'RCO',
'Passengers Transported': 'Passengers',
'Accident Type': 'Accident_Type',
'Persons Evacuated': 'Persons_Evacuated',
'Positive Alcohol Tests': 'Alcohol_Test',
'Positive Drug Tests': 'Drug_Test',
'Hazmat Released Cars': 'Hazmat_Spill_Cars',
'Total Persons Killed': 'Lives_Lost',
'Gross Tonnage': 'Tonnage'}, inplace=True)

### Dropping passenger rail incidents

In [51]:
df.drop(df.loc[df['Equipment']=='2'].index, inplace=True)
df.drop(df.loc[df['Equipment']=='3'].index, inplace=True)
df.drop(df.loc[df['Equipment']=='B'].index, inplace=True)
df.drop(df.loc[df['Equipment']=='C'].index, inplace=True)
df.drop(df.loc[df['Equipment']=='EMU'].index, inplace=True)
df.drop(df.loc[df['Equipment']=='DMU'].index, inplace=True)

### Only needed Equipment column for passenger vs freight

In [52]:
df = df.drop('Equipment', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211276 entries, 1 to 215848
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Railroad           211273 non-null  object 
 1   Date               211274 non-null  object 
 2   Accident_Type      211274 non-null  object 
 3   Hazmat_Spill_Cars  211275 non-null  float64
 4   Persons_Evacuated  211275 non-null  float64
 5   State              211275 non-null  object 
 6   Weather            210882 non-null  object 
 7   Speed              211273 non-null  float64
 8   Tonnage            211275 non-null  float64
 9   RCO                51393 non-null   object 
 10  Alcohol_Test       75718 non-null   float64
 11  Drug_Test          76081 non-null   float64
 12  Passengers         200633 non-null  object 
 13  Damage             211275 non-null  float64
 14  Cause              210178 non-null  object 
 15  Lives_Lost         211275 non-null  float64
dtypes:

### Any rows with NaN for railroad name or date is a error

In [53]:
df.dropna(subset=['Railroad', 'Date'], inplace=True)

### Fill NaN with usable values

In [54]:
df.Weather = df.Weather.fillna('Unknown')
df.Cause = df.Cause.fillna('Unknown')
df.Passengers = df.Passengers.fillna(0)
df.Accident_Type = df.Accident_Type.fillna('Unknown')
df.RCO = df.RCO.fillna(0)
df.Speed = df.Speed.fillna(0)
df.Alcohol_Test = df.Alcohol_Test.fillna(0)
df.Drug_Test = df.Drug_Test.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211272 entries, 1 to 215848
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Railroad           211272 non-null  object 
 1   Date               211272 non-null  object 
 2   Accident_Type      211272 non-null  object 
 3   Hazmat_Spill_Cars  211272 non-null  float64
 4   Persons_Evacuated  211272 non-null  float64
 5   State              211272 non-null  object 
 6   Weather            211272 non-null  object 
 7   Speed              211272 non-null  float64
 8   Tonnage            211272 non-null  float64
 9   RCO                211272 non-null  object 
 10  Alcohol_Test       211272 non-null  float64
 11  Drug_Test          211272 non-null  float64
 12  Passengers         211272 non-null  object 
 13  Damage             211272 non-null  float64
 14  Cause              211272 non-null  object 
 15  Lives_Lost         211272 non-null  float64
dtypes:

### Reset index after dropped rows

In [55]:
df = df.reset_index(drop=True)
df.index

RangeIndex(start=0, stop=211272, step=1)

### Change data types

In [56]:
df = df.astype({'Tonnage': 'Int64', 'Damage': 'Int64', 
'Lives_Lost': 'Int64', 'Drug_Test': 'Int64', 'Speed': 'Int64',
'Alcohol_Test': 'Int64', 'Hazmat_Spill_Cars': 'Int64',
'Persons_Evacuated': 'Int64'})
df['Date'] = df['Date'].astype('datetime64[ns]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211272 entries, 0 to 211271
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Railroad           211272 non-null  object        
 1   Date               211272 non-null  datetime64[ns]
 2   Accident_Type      211272 non-null  object        
 3   Hazmat_Spill_Cars  211272 non-null  Int64         
 4   Persons_Evacuated  211272 non-null  Int64         
 5   State              211272 non-null  object        
 6   Weather            211272 non-null  object        
 7   Speed              211272 non-null  Int64         
 8   Tonnage            211272 non-null  Int64         
 9   RCO                211272 non-null  object        
 10  Alcohol_Test       211272 non-null  Int64         
 11  Drug_Test          211272 non-null  Int64         
 12  Passengers         211272 non-null  object        
 13  Damage             211272 non-null  Int64   

In [57]:
df

,Railroad,Date,Accident_Type,Hazmat_Spill_Cars,Persons_Evacuated,State,Weather,Speed,Tonnage,RCO,Alcohol_Test,Drug_Test,Passengers,Damage,Cause,Lives_Lost
0,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,2017-06-06,Derailment,0,0,ILLINOIS,Clear,0,0,0,0,0,0,0,Switch point worn or broken,0
1,Conrail,1981-04-06,Side collision,0,0,NEW YORK,Snow,0,0,0,0,0,No,4000,Cars left foul,0
2,Illinois Central Railroad Company,2007-01-11,Derailment,0,0,LOUISIANA,Cloudy,4,14100,0.0,0,0,No,5220,Switch damaged or out of adjustment,0
3,Illinois Central Railroad Company,2017-10-07,Hwy-rail crossing,0,0,ILLINOIS,Clear,0,3482,0.0,0,0,No,13000,Highway user deliberately disregarded crossing...,0
4,Norfolk Southern Railway Company,2017-10-15,Derailment,0,0,ILLINOIS,Rain,3,6706,0.0,0,0,No,7418,"Shoving movement, absence of man on or at lead...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211267,Alton & Southern Railway,2017-06-21,Raking collision,0,0,ILLINOIS,Clear,3,0,1.0,0,0,No,13130,Yard skate slid and failed to stop cars,0
211268,Union Pacific Railroad Company,2016-12-13,Derailment,0,0,ILLINOIS,Cloudy,9,0,0.0,0,0,No,36889,Failure to comply with restricted speed or its...,0
211269,Union Pacific Railroad Company,2016-12-13,Derailment,0,0,ILLINOIS,Cloudy,5,450,0.0,0,0,No,24378,Failure to comply with restricted speed or its...,0
211270,Union Pacific Railroad Company,2017-02-01,Derailment,0,0,ILLINOIS,Cloudy,30,563,0.0,0,0,No,59262,Failure to release hand brakes on car(s) (rail...,0


In [58]:
df.groupby('Weather')['Weather'].count()

Weather
Clear      134689
Cloudy      52719
Fog          3081
Rain        14865
Sleet         409
Snow         5116
Unknown       393
Name: Weather, dtype: int64

In [59]:
df.groupby('Lives_Lost')['Lives_Lost'].count()

Lives_Lost
0     208806
1       1956
2        329
3         90
4         44
5         14
6          9
7          7
8          4
9          2
10         2
11         3
16         3
25         1
47         2
Name: Lives_Lost, dtype: int64

In [60]:
df.State.value_counts()

ILLINOIS                20756
TEXAS                   20755
CALIFORNIA              10377
PENNSYLVANIA             9015
OHIO                     8666
IOWA                     7314
MINNESOTA                6651
MISSOURI                 6537
NEW YORK                 6500
INDIANA                  6327
NEBRASKA                 6065
KANSAS                   5839
GEORGIA                  5809
LOUISIANA                5675
TENNESSEE                5190
WISCONSIN                4717
ALABAMA                  4563
KENTUCKY                 4555
WASHINGTON               4481
MICHIGAN                 4459
ARKANSAS                 4337
OKLAHOMA                 3718
VIRGINIA                 3676
FLORIDA                  3391
OREGON                   3078
MISSISSIPPI              3070
COLORADO                 3057
WYOMING                  2872
WEST VIRGINIA            2808
NORTH CAROLINA           2802
NEW JERSEY               2720
MONTANA                  2663
MARYLAND                 2169
NORTH DAKO

In [65]:
df.groupby(['Passengers', 'State'])['Railroad'].count()

Passengers  State        
0           ALABAMA          470
            ALASKA            18
            ARIZONA           24
            ARKANSAS         105
            CALIFORNIA       753
                            ... 
Yes         VIRGINIA          42
            WASHINGTON        35
            WEST VIRGINIA     29
            WISCONSIN         18
            WYOMING            3
Name: Railroad, Length: 150, dtype: int64

In [62]:
df.Passengers.value_counts()

No     197341
0       10642
Yes      3289
Name: Passengers, dtype: int64

In [63]:
df.Speed.value_counts()

0      31036
5      30767
4      20171
10     17867
3      16320
       ...  
123        1
99         1
83         1
86         1
87         1
Name: Speed, Length: 99, dtype: Int64

In [64]:
df

,Railroad,Date,Accident_Type,Hazmat_Spill_Cars,Persons_Evacuated,State,Weather,Speed,Tonnage,RCO,Alcohol_Test,Drug_Test,Passengers,Damage,Cause,Lives_Lost
0,NORTHEAST ILLINOIS REGIONAL COMMUTER RAIL CORP...,2017-06-06,Derailment,0,0,ILLINOIS,Clear,0,0,0,0,0,0,0,Switch point worn or broken,0
1,Conrail,1981-04-06,Side collision,0,0,NEW YORK,Snow,0,0,0,0,0,No,4000,Cars left foul,0
2,Illinois Central Railroad Company,2007-01-11,Derailment,0,0,LOUISIANA,Cloudy,4,14100,0.0,0,0,No,5220,Switch damaged or out of adjustment,0
3,Illinois Central Railroad Company,2017-10-07,Hwy-rail crossing,0,0,ILLINOIS,Clear,0,3482,0.0,0,0,No,13000,Highway user deliberately disregarded crossing...,0
4,Norfolk Southern Railway Company,2017-10-15,Derailment,0,0,ILLINOIS,Rain,3,6706,0.0,0,0,No,7418,"Shoving movement, absence of man on or at lead...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211267,Alton & Southern Railway,2017-06-21,Raking collision,0,0,ILLINOIS,Clear,3,0,1.0,0,0,No,13130,Yard skate slid and failed to stop cars,0
211268,Union Pacific Railroad Company,2016-12-13,Derailment,0,0,ILLINOIS,Cloudy,9,0,0.0,0,0,No,36889,Failure to comply with restricted speed or its...,0
211269,Union Pacific Railroad Company,2016-12-13,Derailment,0,0,ILLINOIS,Cloudy,5,450,0.0,0,0,No,24378,Failure to comply with restricted speed or its...,0
211270,Union Pacific Railroad Company,2017-02-01,Derailment,0,0,ILLINOIS,Cloudy,30,563,0.0,0,0,No,59262,Failure to release hand brakes on car(s) (rail...,0
